In [ ]:
print("Start")

Start


Breakdown of Each Library

1️⃣ accelerate – Hugging Face ka library jo multi-GPU, TPU & distributed training ko optimize karta hai.

Agar FSDP, DeepSpeed use kar rahe ho toh must-have hai.

2️⃣ peft (Parameter Efficient Fine-Tuning) –

LoRA, QLoRA, Adapters jaise low-memory tuning methods ke liye use hota hai.
Full fine-tuning ki jagah lightweight & efficient tuning karne me madad karta hai.

3️⃣ bitsandbytes –

8-bit aur 4-bit quantization support karta hai.
QLoRA fine-tuning me VRAM kaafi save hota hai.

4️⃣ git+https://github.com/huggingface/transformers –

Hugging Face ke transformers ka latest GitHub version install karta hai.
Ye zaroori hai agar koi naye models ya features chahiye ho jo PyPI version me nahi mile.

5️⃣ trl (Transformer Reinforcement Learning) –

RLHF (Reinforcement Learning from Human Feedback) ke liye.
Agar ChatGPT-like models banana hai toh `trl`` ka use hota hai.

6️⃣ py7zr –

7z format wali compressed files ko unzip karne ke liye.
Agar Hugging Face ya kisi aur se compressed dataset mila toh ye useful hoga.

7️⃣ auto-gptq –

GPTQ-based quantization ke liye.
Faster inference aur VRAM efficiency improve karta hai.

8️⃣ optimum –

Hugging Face ka library jo ONNX, TensorRT, Habana Gaudi, NeuronX jaise hardware optimizations provide karta hai.

Accelerated inference aur optimized training ke liye best hai.

🔥 Summary

Agar low-VRAM GPUs (24GB ya less) par fine-tuning kar rahe ho toh bitsandbytes + peft + QLoRA combo best hai.

Agar multi-GPU/TPU cluster pe train kar rahe ho toh accelerate + optimum zaroori hai.

Agar RLHF (like ChatGPT) fine-tune karna hai toh TRL package kaam aayega.

In [ ]:
!pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-zrsw5wzs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-zrsw5wzs
  Resolved https://github.com/huggingface/transformers to commit 41925e42135257361b7f02aa20e3bbdab3f7b923
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 41.7 MB/s eta

 Breakdown
1️⃣ from huggingface_hub import notebook_login

This imports the notebook_login function, which is used for authentication inside Jupyter Notebooks or Google Colab.

2️⃣ notebook_login()

This will prompt you to enter your Hugging Face access token.
You can get the token from Hugging Face website.

🔥 Why is this important?

If you are downloading a private model or dataset, authentication is required.

If you want to upload your fine-tuned model back to Hugging Face, you need to log in first.

💡 Alternative for Script-Based Login

If you are running a script (not in a notebook), use:

from huggingface_hub import login

login(token="your_huggingface_token")


In [ ]:
from huggingface_hub import login
import os

hf_token =
if hf_token:
    login(token=hf_token)
    print("Logged in successfully!")
else:
    print("Error: Hugging Face token not found. Please set the 'HUGGINGFACE_TOKEN' environment variable.")


Logged in successfully!


In [ ]:
import torch

In [ ]:
from datasets import load_dataset , Dataset

In [ ]:
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
data = load_dataset("samsum", split="train")

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
data.to_pandas().head(2)

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...


###Human: Summarize this following dialogue:  

Amanda: I baked cookies. Do you want some?  
Jerry: Yes, I’d love some.  

###Assistant:

Amanda baked cookies and will bring Jerry some.

In [ ]:
data_df = data.to_pandas()

In [ ]:
data_df["text"] = data_df[["dialogue", "summary"]].apply(lambda x: "###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " +x["summary"], axis=1)

In [ ]:
data_df.head(2)

,id,dialogue,summary,text
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...,###Human: Summarize this following dialogue: A...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...,###Human: Summarize this following dialogue: O...


💡 Bottom Line:

NER → Each token is classified separately (input-output separate).
Generation → Model generates text sequentially, so merging helps provide proper context.

In [ ]:
from transformers import BitsAndBytesConfig


In [ ]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '675751e35baa5036fd5f3f9b', 'name': 'abhijeetalande12', 'fullname': 'Abhijeet Alande', 'email': 'abhijeetalande12@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/d4d3623a103326471cbf158ebe6c37a3.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'genai', 'role': 'read', 'createdAt': '2025-02-08T15:21:09.858Z'}}}


# quantization Model

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,    # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    quantization_config=quantization_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

# Define LORA

In [ ]:
peft_config = LoraConfig(
        r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
    )

QLORA

In [ ]:
model = get_peft_model(model , peft_config)

In [ ]:
training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-samsum",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        push_to_hub=True,
        report_to="none"
  )

In [ ]:
# Create the SFTTrainer
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        args=training_arguments,
        tokenizer=tokenizer,
    )

In [ ]:
trainer.train()

# compute matrix is for validation only

# ❓ Why Don’t We Use DataCollator in Generation?
No Label Padding Needed

In classification tasks, DataCollator helps pad labels to ensure tensors are the same size.
But in generation, outputs (responses) are of different lengths, so padding is handled differently (inside the model).

Here No label also For causal language models (like Mistral), padding is usually less critical because models can handle variable-length sequences using left-padding.

# If you pass your entire dataset with all four columns (id, dialogue, summary, text) into a Causal Language Model (CLM) like Mistral, it will only process the text field by default.

# 2. Why No map on Data?
The .map() function in Hugging Face Datasets is used to apply transformations like tokenization or preprocessing.
If you're using BERT, .map() is typically used to tokenize text and create required fields (input_ids, attention_mask, etc.).
If you're using Mistral (or any decoder-only model like GPT-3, LLaMA, etc.), you mainly need tokenized text and might not require .map() for additional fields.

# This Model want Promt where bert Input output

In [ ]:
trainer.push_to_hub()

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-samsum")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-samsum",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

In [ ]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: Sunny: I'm at the railway station in Chennai Karthik: No problems so far? Sunny: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")

In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Thebloke

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

In [ ]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
                          "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                          quantization_config=quantization_config_loading,
                          device_map="auto")

# quantization -> Higher bit to lower bit

onebitLLM paper